# Actividad 1:
# Explorando la estructura oculta de los datos con clustering jerárquico y reducción de dimensionalidad

## Objetivo
Aplicar técnicas de clustering jerárquico aglomerativo a un conjunto de datos sin etiqueta, analizar la estructura de agrupamiento y visualizar los resultados mediante PCQ y T-SNE, evaluando su efectividad como herramientas exploratorias en aprendizaje no supervisado.

**Datasets utilizados:**  
`Wine`

---

### Estructura del Notebook:
1. Metodología.
2. Configuración del entorno.
3. Definicion de funciones.
4. Uso de funciones y resultados.
5. Análisis de los resultados y reflexiones finales.

---

## 1. Metodología

### Flujo de trabajo

1. **Carga y preprocesamiento de datos:**
    - Se carga el dataset **Wine** y se escala.

2. **Clustering y reducción de dimensionalidad:**
    - Se aplica clustering mediante **linkage** con y sin PCA, se generan dos agrupaciones con 2 y 3 clusters y se realiza reducción usando PCA y TSNE. 
    - Las técnicas de reducción de dimensionalidad son exclusivamente para la visualización posterior al clustering, según lo indicado en las instrucciones.

3. **Visualización e interpretación:**
    - Se genera un resumen donde se muestran los resultados de las agrupaciones con 2 y 3 clusters, incluyendo el conteo de muestras por cluster y el Silhouette Score. Se incluyen además la varianza explicada por PCA y la varianza total explicada para el PCA con 2 componentes
    - Se crean dos figuras con el dendrograma para mostrar el clustering jerárquico. Cada figura tiene dos dendrogramas, la primera figura se genera con un Z sin aplicar PCA, mientras que la segunda es después de aplicar PCA.
    - Se generan 2 figuras, cada una con 2 gráficos, que muestran la distribución de los componentes principales y componentes t-SNE para 2 y 3 clusters.

---

# 2. Configuración del entorno

--- 

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
from sklearn.datasets import load_wine
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# 3. Definición de funciones

> **Nota:** Para mejor comprensión de las funciones y su utilidad, esta sección se divide en bloques, en donde cada uno responde a una parte diferente de la metodología de trabajo. 

---

**Bloque 1:** Carga y preprocesamiento de datos.

- **`carga_y_preprocesamiento()`** 
Carga y escala el dataset.

---

`Justificacion del escalado y la no división de los datos:`

Para este trabajo no se realizo una limpieza exaustiva de datos, solo se escalaron debido a que como técnicas como el PCA trabajan con la varianza, tener variables con valores o rangos de valores muy amplios produciria un mal funcionamiento del modelo.

En esta etapa del análisis, donde se aplican técnicas no supervisadas como clustering jerárquico y reducción de dimensionalidad (por ejemplo, PCA o t-SNE), dado que no se cuenta con una variable objetivo, no es necesario realizar una separación entre entrenamiento y prueba. Estas técnicas buscan explorar la estructura interna de los datos, identificar patrones o visualizar relaciones, por lo que se benefician del uso del conjunto completo para preservar la distribución global.

No obstante, si estas transformaciones se utilizaran posteriormente como parte de un modelo supervisado, sería fundamental aplicar primero una separación (train_test_split), ajustar las transformaciones solo con los datos de entrenamiento, y luego aplicarlas al conjunto de prueba. Esto evita la fuga de información (data leakage) y garantiza una evaluación justa del rendimiento del modelo.

---

In [ ]:
def carga_y_preprocesamiento():
    """
    Carga el dataset Wine desde scikit-learn y aplica escalado estándar a las variables numéricas.

    Returns:
        X (numpy.ndarray): Matriz de características original sin escalar.
        X_scaled (numpy.ndarray): Matriz de características escalada con media 0 y desviación estándar 1.
        feature_names (list): Lista con los nombres de las características del dataset.

    Nota:
        El escalado se realiza usando StandardScaler, útil para normalizar variables antes de aplicar modelos sensibles a la escala.
    """
    data = load_wine()
    X = data.data
    feature_names = data.feature_names
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    return X, X_scaled, feature_names

**Bloque 2:** Clustering y reducción de dimensionalidad.

- **`clustering_y_reduccion()`** 
Realiza clustering jerárquico.

---

`Justificacion de la estructura de la función:`

Aunque t-SNE no se utiliza para crear clústeres sino para visualización, en este flujo se aplican tanto PCA como t-SNE antes de generar las visualizaciones, y no como parte del proceso de clustering en sí. Esta decisión responde a una cuestión de modularidad y claridad: la función clustering_y_reduccion encapsula todo el preprocesamiento necesario (reducción dimensional + clustering jerárquico con diferentes cantidades de clústeres), de modo que los resultados puedan ser fácilmente reutilizados por funciones de visualización. Así, mantenemos la separación de responsabilidades: una función produce los datos necesarios para analizar y graficar los clústeres, y otra se encarga de mostrarlos. Esta estructura también permite comparar los agrupamientos bajo diferentes perspectivas (PCA y t-SNE) sin repetir lógica ni romper el principio de responsabilidad única.

---

In [ ]:
def clustering_y_reduccion(X_scaled, t_values=[2,3], random_state=42):
    """
    Realiza clustering jerárquico con enlace Ward, segmenta el dendrograma en distintos números de clusters,
    y aplica reducción dimensional con PCA y t-SNE sobre los datos escalados.

    Args:
        X_scaled (numpy.ndarray): Matriz de características ya escalada.
        t_values (list, optional): Lista de enteros que indica los números de clusters para cortar el dendrograma. Por defecto [2, 3].
        random_state (int, optional): Semilla para reproducibilidad en t-SNE. Por defecto 42.

    Returns:
        Z (numpy.ndarray): Matriz de linkage resultante del clustering jerárquico antes de aplicar PCA.
        Z_pca (numpy.ndarray): Matriz de linkage resultante del clustering jerárquico después de aplicar PCA.
        clusters_dict (dict): Diccionario donde las claves son los valores de t y los valores son arrays con etiquetas de cluster para cada punto.
        X_pca (numpy.ndarray): Proyección de los datos en 2 componentes principales.
        X_tsne (numpy.ndarray): Proyección de los datos en 2 dimensiones mediante t-SNE.
        explained_variance (numpy.ndarray): Porcentaje de varianza explicada por cada componente principal en PCA.
    """
    # Linkage
    Z = linkage(X_scaled, method='ward')
    
    # Crear clusters para cada t en t_values
    clusters_dict = {}
    for t in t_values:
        clusters = fcluster(Z, t, criterion='maxclust')
        clusters_dict[t] = clusters
    
    # PCA
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X_scaled)
    explained_variance = pca.explained_variance_ratio_

    # Linkage post PCA
    Z_pca = linkage(X_pca, method='ward')
    
    # t-SNE
    tsne = TSNE(n_components=2, random_state=random_state)
    X_tsne = tsne.fit_transform(X_scaled)
    
    return Z, Z_pca, clusters_dict, X_pca, X_tsne, explained_variance

**Bloque 3:** Generacion y visualización de resultados.

- **`mostrar_resumen()`** 
Genera un resumen de la informacion de las agrupaciones en 2 y 3 clusters con el número de muestras por cada cluster y el Silhouette Score por agrupación.

- **`graficar_dendrogramas()`** 
Genera dos dendrogramas sin aplicar PCA, el primero se muestra sin limitaciones, mientras que el segundo esta truncado y limitado a los últimos 20 clusters.

- **`graficar_dendrogramas_post_pca()`** 
Genera dos dendrogramas después de aplicar PCA, el primero se muestra sin limitaciones, mientras que el segundo esta truncado y limitado a los últimos 20 clusters.

- **`graficar_clusters()`** 
Grafica los resultados de PCA y TNSE para las agrupaciones de 2 y 3 clusters, generando dos gráficas por figura..

---

`Justificacion de la elección de visualizaciones:`

Se incluyeron visualizaciones específicas para facilitar la interpretación de los resultados del clustering jerárquico y su calidad. La función mostrar_resumen() entrega un panorama cuantitativo de cada agrupamiento (2 y 3 clústeres), mostrando el número de observaciones por clúster y su respectivo Silhouette Score, lo que permite evaluar la cohesión y separación entre clústeres. Por otro lado, graficar_dendrograma_dual() genera dos vistas complementarias del dendrograma: una completa y otra truncada, lo que permite tanto una visión general de la jerarquía global como una inspección detallada de las divisiones finales. Finalmente, graficar_clusters() compara visualmente los resultados de los agrupamientos sobre proyecciones en 2D generadas por PCA y t-SNE, facilitando la identificación de patrones y validando visualmente la estructura propuesta por los clústeres.

---

In [ ]:
def mostrar_resumen(X_scaled, clusters_dict, explained_variance):
    """
    Muestra un resumen estadístico y métrico del clustering realizado.

    Para cada número de clusters especificado en `clusters_dict`, imprime:
    - Los IDs de clusters asignados.
    - El conteo de muestras por cluster.
    - El Silhouette Score que mide la calidad de la agrupación.

    Finalmente, muestra la varianza explicada por las dos primeras componentes del PCA.

    Args:
        X_scaled (numpy.ndarray): Datos escalados usados para clustering.
        clusters_dict (dict): Diccionario con número de clusters como llave y arrays de etiquetas como valores.
        explained_variance (numpy.ndarray): Varianza explicada por cada componente principal en PCA.
    """
    for t, clusters in clusters_dict.items():
        print(f"\n=== Resultados para {t} clusters ===")
        print("Clusters asignados:", np.unique(clusters))
        
        counts = pd.Series(clusters).value_counts().sort_index()
        print("Datos por cluster:")
        for cluster_id, count in counts.items():
            print(f"  Cluster {cluster_id}: {count} muestras")
        
        score = silhouette_score(X_scaled, clusters)
        print(f"Silhouette Score cluster (t={t}): {score:.4f}")
    
    print("\nVarianza explicada por PCA (2 componentes):", explained_variance)
    print("Varianza total explicada:", explained_variance.sum())

def graficar_dendrogramas(Z):
    """
    Genera dos gráficos de dendrogramas basados en la matriz de linkage `Z`:

    - Dendrograma completo que muestra todas las fusiones.
    - Dendrograma truncado mostrando solo los últimos 20 clusters para mejor legibilidad.

    Args:
        Z (numpy.ndarray): Matriz de linkage generada por clustering jerárquico antes de aplicar PCA.
    """
    fig, axes = plt.subplots(1, 2, figsize=(18, 6))

    # Dendrograma completo
    axes[0].set_title('Dendrograma completo')
    dendrogram(Z, ax=axes[0])
    axes[0].set_xlabel('Índice de muestra')
    axes[0].set_ylabel('Distancia')

    # Dendrograma truncado
    axes[1].set_title('Dendrograma truncado (últimos 20 clusters)')
    dendrogram(
        Z,
        ax=axes[1],
        truncate_mode="lastp",
        p=20,
        leaf_rotation=90.,
        leaf_font_size=10,
        show_contracted=True
    )
    axes[1].set_xlabel('Cluster')
    axes[1].set_ylabel('Distancia')

    plt.tight_layout()
    plt.show()

def graficar_dendrogramas_post_pca(Z_pca):
    """
    Genera dos gráficos de dendrogramas basados en la matriz de linkage `Z` después de aplicar PCA:

    - Dendrograma completo que muestra todas las fusiones.
    - Dendrograma truncado mostrando solo los últimos 20 clusters para mejor legibilidad.

    Args:
        Z_PCA (numpy.ndarray): Matriz de linkage generada por clustering jerárquico DESPUE´S DE APLICAR pca.
    """
    fig, axes = plt.subplots(1, 2, figsize=(18, 6))

    # Dendrograma completo
    axes[0].set_title('Dendrograma completo')
    dendrogram(Z_pca, ax=axes[0])
    axes[0].set_xlabel('Índice de muestra')
    axes[0].set_ylabel('Distancia')

    # Dendrograma truncado
    axes[1].set_title('Dendrograma truncado (últimos 20 clusters)')
    dendrogram(
        Z_pca,
        ax=axes[1],
        truncate_mode="lastp",
        p=20,
        leaf_rotation=90.,
        leaf_font_size=10,
        show_contracted=True
    )
    axes[1].set_xlabel('Cluster')
    axes[1].set_ylabel('Distancia')

    plt.tight_layout()
    plt.show()

def graficar_clusters(X_pca, X_tsne, clusters_dict):
    """
    Grafica la asignación de clusters sobre las representaciones reducidas con PCA y t-SNE.

    Para cada cantidad de clusters en `clusters_dict`, crea dos gráficos de dispersión:
    - Visualización con los dos primeros componentes principales (PCA).
    - Visualización con las dos dimensiones obtenidas por t-SNE.

    Args:
        X_pca (numpy.ndarray): Datos proyectados en las dos primeras componentes principales.
        X_tsne (numpy.ndarray): Datos proyectados en dos dimensiones mediante t-SNE.
        clusters_dict (dict): Diccionario con número de clusters y sus etiquetas asignadas.
    """
    t_values = list(clusters_dict.keys())

    fig, axs = plt.subplots(1, len(t_values), figsize=(7 * len(t_values), 6))
    if len(t_values) == 1:
        axs = [axs]

    for ax, t in zip(axs, t_values):
        cluster_ids = clusters_dict[t]
        
        # Crear DataFrame para seaborn
        df_plot = pd.DataFrame({
            'PC1': X_pca[:, 0],
            'PC2': X_pca[:, 1],
            'Grupo': cluster_ids.astype(str)  # Opcional: convertir a texto tipo '0', '1', etc.
        })
        
        sns.scatterplot(
            data=df_plot,
            x='PC1',
            y='PC2',
            hue='Grupo',
            palette='Set1',
            ax=ax,
            edgecolor='k',
            s=60
        )
        
        ax.set_title(f'Clustering Jerárquico ({t} clusters) visualizado con PCA')
        ax.set_xlabel('PC1')
        ax.set_ylabel('PC2')
        ax.legend(title='Grupo')

    plt.tight_layout()
    plt.show()

    fig, axs = plt.subplots(1, len(t_values), figsize=(7 * len(t_values), 6))
    if len(t_values) == 1:
        axs = [axs]

    for ax, t in zip(axs, t_values):
        cluster_ids = clusters_dict[t]
        
        # Crear DataFrame para seaborn
        df_plot = pd.DataFrame({
            'Componente1': X_tsne[:, 0],
            'Componente2': X_tsne[:, 1],
            'Grupo': cluster_ids.astype(str)  # Opcional: convertir a texto tipo '0', '1'...
        })
        
        sns.scatterplot(
            data=df_plot,
            x='Componente1',
            y='Componente2',
            hue='Grupo',
            palette='Set1',
            ax=ax,
            edgecolor='k',
            s=60
        )
        
        ax.set_title(f'Clustering Jerárquico ({t} clusters) visualizado con t-SNE')
        ax.set_xlabel('Componente 1')
        ax.set_ylabel('Componente 2')
        ax.legend(title='Grupo')

    plt.tight_layout()
    plt.show()

**Bloque 4:** Función de ejecución del código.

- **`main()`** 
Ejecuta el flujo completo de análisis y visualización de clustering jerárquico.

In [ ]:
def main():
    """
    Ejecuta el flujo completo de análisis y visualización de clustering jerárquico sobre el dataset Wine.

    Pasos realizados:
    1. Carga y preprocesamiento de los datos (escalado).
    2. Clustering jerárquico con método Ward y reducción dimensional con PCA y t-SNE.
    3. Muestra un resumen numérico y métrico de los clusters encontrados.
    4. Grafica dendrogramas completos y truncados para interpretación visual.
    5. Visualiza los clusters sobre las proyecciones PCA y t-SNE.

    No recibe parámetros ni retorna valores, ya que su función es orquestar el proceso completo.
    """
    X, X_scaled, feature_names = carga_y_preprocesamiento()
    Z, Z_pca, clusters_dict, X_pca, X_tsne, explained_variance = clustering_y_reduccion(X_scaled)
    mostrar_resumen(X_scaled, clusters_dict, explained_variance)

    print("\nDendrograma sin PCA")
    graficar_dendrogramas(Z)

    print("\nDendrograma después de PCA")
    graficar_dendrogramas_post_pca(Z_pca)

    print("\nGráficos de clusters con PCA y t-SNE")
    graficar_clusters(X_pca, X_tsne, clusters_dict)

# 4. Visualización de resultados

Se muestran los resultados obtenidos a partir de la ejecución de la funcion **main()**.

---

In [ ]:
if __name__ == "__main__":
    main()

# 5. Análisis de los resultados y reflexiones finales

---

## Análisis del dendrograma: ¿Reflejan los grupos alguna estructura evidente?

El análisis de clustering jerárquico mediante enlace completo revela una estructura de agrupamiento distintiva en los datos. Observando el "Dendrograma completo" (izquierda), se manifiesta una jerarquía donde las uniones de las ramas a diferentes alturas señalan la disimilitud entre clústeres; uniones más elevadas implican mayores diferencias entre los grupos fusionados. La visualización truncada a la derecha, "Dendrograma truncado (últimos 20 clusters)", resalta las divisiones más significativas, facilitando la identificación de macrogrupos. Se distinguen notablemente tres grandes conglomerados que se forman a una distancia superior a ~25 en el eje vertical. Esto subraya la identificación de una jerarquía sustancial en los datos, donde ciertos grupos presentan homogeneidad interna y diferenciación externa.

En contraste, al examinar los dendrogramas generados después de aplicar Análisis de Componentes Principales (PCA) (como se observa en la segunda figura de dendrogramas), se aprecia una estructura de agrupamiento más simplificada. El dendrograma completo con PCA muestra menos ramificaciones en los niveles inferiores, lo que sugiere que la reducción de dimensionalidad ha disminuido la variabilidad entre las muestras iniciales, haciendo que las agrupaciones tempranas sean menos diferenciadas. Al observar el dendrograma truncado con PCA, se puede notar que los macrogrupos tienden a formarse a distancias menores en comparación con el dendrograma sin PCA. Esto indica que, después de la reducción de dimensionalidad, las diferencias entre los principales conglomerados se vuelven más pronunciadas a una escala de distancia menor. La aplicación de PCA, al enfocarse en las direcciones de mayor varianza, parece haber consolidado las agrupaciones, presentando una visión más concisa de la estructura de alto nivel en los datos. Esta comparación subraya cómo la reducción de dimensionalidad a través de PCA puede influir en la percepción de la granularidad y la distancia relativa entre los clústeres identificados por el clustering jerárquico.

Este resultado se refuerza con los análisis y visualizaciones de PCA y t-SNE:

- En el PCA (Análisis de Componentes Principales), al proyectar los datos en dos dimensiones, se observan agrupaciones visibles pero parcialmente solapadas, lo cual sugiere que existe cierta estructura latente en las variables, aunque con una separación no completamente lineal.

- En el gráfico de t-SNE, que preserva mejor las distancias locales, los grupos se aprecian más separados y definidos, confirmando la existencia de patrones subyacentes bien diferenciados entre subconjuntos de observaciones.

En conjunto, estas tres visualizaciones respaldan la conclusión de que los datos presentan una estructura interna con agrupamientos coherentes, lo que valida la aplicación de técnicas no supervisadas para su exploración.

---

## Comparación entre PCA y T-SNE

Diferencias metodológicas

- PCA es una técnica lineal: proyecta los datos en las direcciones de máxima varianza. Es útil para estructuras globales, pero puede perder relaciones locales o no lineales.

- t-SNE es una técnica no lineal: prioriza preservar la estructura local, es decir, mantiene juntos a los puntos que estaban cerca en el espacio original, aunque no siempre respeta la escala o las distancias globales.

Ambas técnicas, PCA y t-SNE, muestran una separación clara entre los clusters, destacándose que en t-SNE los grupos son aún más compactos y están mejor separados, especialmente cuando se consideran tres clusters. En cuanto a la distribución de los grupos, PCA presenta una estructura más lineal y continua, basada en los ejes de varianza, mientras que t-SNE revela agrupaciones con formas más curvas y naturales, como pequeños "islotes". Respecto a la estructura global, PCA conserva mejor la continuidad y escala original, evidenciándose un eje continuo en ambas direcciones, en contraste con t-SNE donde dicha escala se pierde y los grupos parecen flotar libremente. Finalmente, cuando se analiza la formación de subgrupos internos con tres clusters, t-SNE ofrece una definición más clara y menos solapamiento que PCA, donde los subgrupos son visibles pero tienden a mezclarse un poco más.

Los resultados de PCA y t-SNE son diferentes principalmente por la naturaleza de cada técnica. Mientras que PCA conserva la estructura global de los datos y muestra una separación razonable entre grupos, t-SNE destaca la estructura local, revelando agrupaciones más compactas y claramente separadas. Esto sugiere que la estructura de los datos puede tener relaciones no lineales que t-SNE capta mejor. En nuestro caso, ambas técnicas reflejan la existencia de grupos diferenciados, pero t-SNE permite una mejor visualización de las fronteras entre ellos.

---